In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, train_test_split

In [2]:
import gc
gc.enable()

In [3]:
def train_model(train_, val_x_, val_y_, test_, y_, folds_):
    feats_ = [f_ for f_ in test_.columns if f_ not in ['SK_ID_CURR']]
    oof_preds = np.zeros(train_.shape[0])
    val_preds = np.zeros(val_x_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    

    
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(train_)):
        #print(train_.type)
        trn_x, trn_y = pd.DataFrame(train_).iloc[trn_idx], pd.DataFrame(y_).iloc[trn_idx]
        val_x, val_y = pd.DataFrame(train_).iloc[val_idx], pd.DataFrame(y_).iloc[val_idx]
        
        clf = XGBClassifier(
            min_child_weight = 0.01, 
            learning_rate = 0.3, 
            max_depth = 10, 
            n_estimators = 50, 
            n_jobs = -1,
            gamma = 0.5, 
            subsample = 0.9, 
            colsample_bytree = 0.8, 
            booster = 'gbtree', 
            scale_pos_weight = 2, 
            reg_alpha = 1
            )
        clf.fit(trn_x, trn_y, 
            eval_set = [(trn_x, trn_y), (val_x, val_y)],
            eval_metric = 'auc', verbose =True, early_stopping_rounds = 10)
        
        oof_preds[val_idx] = clf.predict_proba(val_x)[:, 1]
        val_preds += clf.predict_proba(pd.DataFrame(val_x_[feats_]))[:, 1] / folds_.n_splits
        sub_preds += clf.predict_proba(pd.DataFrame(test_[feats_]))[:, 1] / folds_.n_splits
        print('fold %2d validate AUC score %.6f'%(n_fold + 1,roc_auc_score(val_y_, val_preds) * folds_.n_splits))
        print('fold %2d AUC %.6f'%(n_fold+1, roc_auc_score(val_y, oof_preds[val_idx])))
        del clf, trn_x, trn_y, val_x, val_y
        gc.collect()
    print('validate AUC score %.6f'%roc_auc_score(val_y_, val_preds))
    print('full AUC score %.6f'%roc_auc_score(y_, oof_preds))
    test_['TARGET'] = sub_preds
    
    return oof_preds, test_[['SK_ID_CURR', 'TARGET']]

In [4]:
train = pd.read_csv('../data/train_.csv')
test = pd.read_csv('../data/test_.csv')
y = pd.read_csv('../data/y_.csv')
val_x = pd.read_csv('../data/val_x_.csv')
val_y = pd.read_csv('../data/val_y_.csv')

In [5]:
print(train.shape,test.shape,val_x.shape)
#train_x, val_x, train_y, val_y = train_test_split(train, y, test_size=0.1, random_state = 14)
folds = KFold(n_splits=5, shuffle=True, random_state=0)

(508761, 389) (48744, 390) (30752, 390)


In [8]:
y.head()

,Unnamed: 0,0
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0


In [9]:
val_y.head()

,Unnamed: 0,TARGET
0,32413,0
1,226702,0
2,106022,0
3,62854,0
4,232481,0


In [10]:
val_x.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,bur_SK_ID_BUREAU,bur_DAYS_CREDIT,bur_CREDIT_DAY_OVERDUE,bur_DAYS_CREDIT_ENDDATE,bur_DAYS_ENDDATE_FACT,bur_AMT_CREDIT_MAX_OVERDUE,bur_CNT_CREDIT_PROLONG,bur_AMT_CREDIT_SUM,bur_AMT_CREDIT_SUM_DEBT,bur_AMT_CREDIT_SUM_LIMIT,bur_AMT_CREDIT_SUM_OVERDUE,bur_DAYS_CREDIT_UPDATE,bur_AMT_ANNUITY,bur_ca_Active,bur_ca_Bad debt,bur_ca_Closed,bur_ca_Sold,bur_cc_currency 1,bur_cc_currency 2,bur_cc_currency 3,bur_cc_currency 4,bur_ct_Another type of loan,bur_ct_Car loan,bur_ct_Cash loan (non-earmarked),bur_ct_Consumer credit,bur_ct_Credit card,bur_ct_Interbank credit,bur_ct_Loan for business development,bur_ct_Loan for purchase of shares (margin lending),bur_ct_Loan for the purchase of equipment,bur_ct_Loan for working capital replenishment,bur_ct_Microloan,bur_ct_Mobile operator loan,bur_ct_Mortgage,bur_ct_Real estate loan,bur_ct_Unknown type of loan,bur_bur_bal_MONTHS_BALANCE,bur_bur_bal_bur_bal_status_0,bur_bur_bal_bur_bal_status_1,bur_bur_bal_bur_bal_status_2,bur_bur_bal_bur_bal_status_3,bur_bur_bal_bur_bal_status_4,bur_bur_bal_bur_bal_status_5,bur_bur_bal_bur_bal_status_C,bur_bur_bal_bur_bal_status_X,bur_bur_bal_bur_cnt,prev_SK_ID_PREV,prev_AMT_ANNUITY,prev_AMT_APPLICATION,prev_AMT_CREDIT,prev_AMT_DOWN_PAYMENT,prev_AMT_GOODS_PRICE,prev_HOUR_APPR_PROCESS_START,prev_NFLAG_LAST_APPL_IN_DAY,prev_RATE_DOWN_PAYMENT,prev_RATE_INTEREST_PRIMARY,prev_RATE_INTEREST_PRIVILEGED,prev_DAYS_DECISION,prev_SELLERPLACE_AREA,prev_CNT_PAYMENT,prev_DAYS_FIRST_DRAWING,prev_DAYS_FIRST_DUE,prev_DAYS_LAST_DUE_1ST_VERSION,prev_DAYS_LAST_DUE,prev_DAYS_TERMINATION,prev_NFLAG_INSURED_ON_APPROVAL,prev_NAME_CONTRACT_TYPE_Cash loans,prev_NAME_CONTRACT_TYPE_Consumer loans,prev_NAME_CONTRACT_TYPE_Revolving loans,prev_NAME_CONTRACT_TYPE_XNA,prev_WEEKDAY_APPR_PROCESS_START_FRIDAY,prev_WEEKDAY_APPR_PROCESS_START_

In [11]:
oof_preds, test_preds = train_model(train, val_x, val_y['TARGET'].values.ravel(), test, y['0'].values.ravel(), folds)
test_preds.to_csv('../data/xgb_submission.csv', index=False)

F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-auc:0.956117	validation_1-auc:0.950894
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.968353	validation_1-auc:0.963516
[2]	validation_0-auc:0.978755	validation_1-auc:0.974154
[3]	validation_0-auc:0.98005	validation_1-auc:0.975088
[4]	validation_0-auc:0.981193	validation_1-auc:0.975582
[5]	validation_0-auc:0.98256	validation_1-auc:0.976394
[6]	validation_0-auc:0.983536	validation_1-auc:0.97678
[7]	validation_0-auc:0.984305	validation_1-auc:0.976992
[8]	validation_0-auc:0.985276	validation_1-auc:0.97718
[9]	validation_0-auc:0.985967	validation_1-auc:0.977402
[10]	validation_0-auc:0.986645	validation_1-auc:0.97756
[11]	validation_0-auc:0.987314	validation_1-auc:0.977713
[12]	validation_0-auc:0.988087	validation_1-auc:0.977833
[13]	validation_0-auc:0.98852	validation_1-auc:0.9779
[14]	validation_0-auc:0.989255	validation_1-auc:0.978022
[15]	v

F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-auc:0.953291	validation_1-auc:0.95011
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.969663	validation_1-auc:0.966405
[2]	validation_0-auc:0.978609	validation_1-auc:0.975043
[3]	validation_0-auc:0.980612	validation_1-auc:0.976373
[4]	validation_0-auc:0.981706	validation_1-auc:0.976801
[5]	validation_0-auc:0.982939	validation_1-auc:0.977411
[6]	validation_0-auc:0.9837	validation_1-auc:0.977716
[7]	validation_0-auc:0.984878	validation_1-auc:0.977916
[8]	validation_0-auc:0.985707	validation_1-auc:0.978167
[9]	validation_0-auc:0.98644	validation_1-auc:0.978311
[10]	validation_0-auc:0.987099	validation_1-auc:0.978574
[11]	validation_0-auc:0.987821	validation_1-auc:0.978693
[12]	validation_0-auc:0.988405	validation_1-auc:0.97871
[13]	validation_0-auc:0.988924	validation_1-auc:0.978716
[14]	validation_0-auc:0.98947	validation_1-auc:0.978758
[15]

F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-auc:0.951452	validation_1-auc:0.947173
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.970345	validation_1-auc:0.966262
[2]	validation_0-auc:0.978719	validation_1-auc:0.97493
[3]	validation_0-auc:0.980616	validation_1-auc:0.976352
[4]	validation_0-auc:0.981644	validation_1-auc:0.976923
[5]	validation_0-auc:0.982677	validation_1-auc:0.977421
[6]	validation_0-auc:0.983694	validation_1-auc:0.97779
[7]	validation_0-auc:0.984527	validation_1-auc:0.977985
[8]	validation_0-auc:0.985461	validation_1-auc:0.978123
[9]	validation_0-auc:0.986268	validation_1-auc:0.978253
[10]	validation_0-auc:0.986928	validation_1-auc:0.978392
[11]	validation_0-auc:0.987654	validation_1-auc:0.97849
[12]	validation_0-auc:0.988212	validation_1-auc:0.978596
[13]	validation_0-auc:0.988761	validation_1-auc:0.978664
[14]	validation_0-auc:0.989383	validation_1-auc:0.978724
[

F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-auc:0.952882	validation_1-auc:0.949958
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.970862	validation_1-auc:0.967949
[2]	validation_0-auc:0.978825	validation_1-auc:0.975426
[3]	validation_0-auc:0.980869	validation_1-auc:0.976669
[4]	validation_0-auc:0.981885	validation_1-auc:0.977077
[5]	validation_0-auc:0.983033	validation_1-auc:0.977495
[6]	validation_0-auc:0.983989	validation_1-auc:0.977826
[7]	validation_0-auc:0.984977	validation_1-auc:0.978007
[8]	validation_0-auc:0.985672	validation_1-auc:0.978077
[9]	validation_0-auc:0.986451	validation_1-auc:0.97824
[10]	validation_0-auc:0.987267	validation_1-auc:0.978308
[11]	validation_0-auc:0.987815	validation_1-auc:0.978406
[12]	validation_0-auc:0.988563	validation_1-auc:0.978545
[13]	validation_0-auc:0.989071	validation_1-auc:0.97867
[14]	validation_0-auc:0.989622	validation_1-auc:0.978693


F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-auc:0.951332	validation_1-auc:0.94556
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.970203	validation_1-auc:0.965496
[2]	validation_0-auc:0.979124	validation_1-auc:0.974632
[3]	validation_0-auc:0.980999	validation_1-auc:0.976162
[4]	validation_0-auc:0.982012	validation_1-auc:0.976718
[5]	validation_0-auc:0.983108	validation_1-auc:0.977185
[6]	validation_0-auc:0.98411	validation_1-auc:0.977467
[7]	validation_0-auc:0.984821	validation_1-auc:0.977692
[8]	validation_0-auc:0.985633	validation_1-auc:0.977915
[9]	validation_0-auc:0.986357	validation_1-auc:0.978096
[10]	validation_0-auc:0.986995	validation_1-auc:0.978228
[11]	validation_0-auc:0.987732	validation_1-auc:0.978189
[12]	validation_0-auc:0.98855	validation_1-auc:0.978368
[13]	validation_0-auc:0.989143	validation_1-auc:0.978443
[14]	validation_0-auc:0.989781	validation_1-auc:0.9785
[15